In [62]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [63]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ashish/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [64]:
df1 = pd.read_csv('tweets_of_f24_users_with_bert_sentiment.csv')

In [65]:
df2 = pd.read_csv('tweets_of_f24_users_with_lang.csv')

In [66]:
df3 = pd.read_csv('tweets_of_f24_users_with_vader_sentiment.csv')

In [67]:
df1

,userid,account_type,clean_tweet,bert_sentiment_json,bert_clear_expression_conf
0,787405734442958848,bot,"We can put the ""River"" in ""Screwdrivers"" https...","{""percentage"": 90, ""sentiment"": ""Negative""}",90
1,787405734442958848,bot,"You can't extract the ""Pong"" from ""Sponged"" ht...","{""percentage"": 98, ""sentiment"": ""Negative""}",98
2,787405734442958848,bot,"Put the ""Nope"" back in ""Inoperative"" https://t...","{""percentage"": 97, ""sentiment"": ""Negative""}",97
3,787405734442958848,bot,"We can't spell ""Artworks"" without ""Two"" https:...","{""percentage"": 85, ""sentiment"": ""Negative""}",85
4,787405734442958848,bot,"You can be the ""She"" in ""Astonished"" https://t...","{""percentage"": 51, ""sentiment"": ""Positive""}",51
...,...,...,...,...,...
488632,3077047801,bot,A Tribute – Joan Rivers on Fitness http://t.co...,"{""percentage"": 69, ""sentiment"": ""Positive""}",69
488633,3077047801,bot,Live in Orlando area? Want a personal trainer?...,"{""percentage"": 87, ""sentiment"": ""Positive""}",87
488634,3077047801,bot,If you're trying to get fit keep working. You...,"{""percentage"": 83, ""sentiment"": ""Positive""}",83
488635,3077047801,bot,Ready for summer? Cheap cool sunglasses - loo...,"{""percentage"": 96, ""sentiment"": ""Negative""}",96


In [68]:
df2

,userid,account_type,clean_tweet,lang
0,787405734442958848,bot,"We can put the ""River"" in ""Screwdrivers"" https...",en
1,787405734442958848,bot,"You can't extract the ""Pong"" from ""Sponged"" ht...",en
2,787405734442958848,bot,"Put the ""Nope"" back in ""Inoperative"" https://t...",en
3,787405734442958848,bot,"We can't spell ""Artworks"" without ""Two"" https:...",en
4,787405734442958848,bot,"You can be the ""She"" in ""Astonished"" https://t...",en
...,...,...,...,...
488632,3077047801,bot,A Tribute – Joan Rivers on Fitness http://t.co...,en
488633,3077047801,bot,Live in Orlando area? Want a personal trainer?...,en
488634,3077047801,bot,If you're trying to get fit keep working. You...,en
488635,3077047801,bot,Ready for summer? Cheap cool sunglasses - loo...,en


In [69]:
df3

,userid,account_type,clean_tweet,vader_sentiment,vader_label
0,787405734442958848,bot,"We can put the ""River"" in ""Screwdrivers"" https...",0.0000,neutral
1,787405734442958848,bot,"You can't extract the ""Pong"" from ""Sponged"" ht...",0.0000,neutral
2,787405734442958848,bot,"Put the ""Nope"" back in ""Inoperative"" https://t...",0.0000,neutral
3,787405734442958848,bot,"We can't spell ""Artworks"" without ""Two"" https:...",0.0000,neutral
4,787405734442958848,bot,"You can be the ""She"" in ""Astonished"" https://t...",0.0000,neutral
...,...,...,...,...,...
488632,3077047801,bot,A Tribute – Joan Rivers on Fitness http://t.co...,0.2732,positive
488633,3077047801,bot,Live in Orlando area? Want a personal trainer?...,0.1680,positive
488634,3077047801,bot,If you're trying to get fit keep working. You...,0.8608,positive
488635,3077047801,bot,Ready for summer? Cheap cool sunglasses - loo...,0.7430,positive


In [85]:
df4 = pd.merge(df1, df2, left_index=True, right_index=True)

In [86]:
df4.columns

Index(['userid_x', 'account_type_x', 'clean_tweet_x', 'bert_sentiment_json',
       'bert_clear_expression_conf', 'userid_y', 'account_type_y',
       'clean_tweet_y', 'lang'],
      dtype='object')

In [87]:
df4 = df4[['userid_x', 'account_type_x', 'clean_tweet_x', 'bert_clear_expression_conf', 'lang']]
df4.columns = ['userid', 'account_type', 'clean_tweet', 'bert_clear_expression_conf', 'lang']

In [89]:
df4.columns

Index(['userid', 'account_type', 'clean_tweet', 'bert_clear_expression_conf',
       'lang'],
      dtype='object')

In [90]:
df4.columns = ['userid', 'account_type', 'clean_tweet', 'bert_clear_expression_conf', 'lang']

In [92]:
df5 = pd.merge(df4, df3, left_index=True, right_index=True)

In [94]:
df5.columns

Index(['userid_x', 'account_type_x', 'clean_tweet_x',
       'bert_clear_expression_conf', 'lang', 'userid_y', 'account_type_y',
       'clean_tweet_y', 'vader_sentiment', 'vader_label'],
      dtype='object')

In [96]:
df5 = df5[['userid_x', 'account_type_x', 'clean_tweet_x', 'bert_clear_expression_conf', 'lang', 'vader_sentiment', 'vader_label']]
df5.columns = ['userid', 'account_type', 'clean_tweet', 'bert_clear_expression_conf', 'lang', 'vader_sentiment', 'vader_label']

In [101]:
def get_url_flag(in_tweet):
    m = re.search(r"http[a-zA-Z0-9/\-.:%]+", in_tweet)
    rtn = False
    if m:
        rtn = True
    return rtn

In [102]:
%%time
df5['url_flag'] = df5['clean_tweet'].apply(get_url_flag)

CPU times: user 855 ms, sys: 7.75 ms, total: 863 ms
Wall time: 860 ms


In [105]:
df5['len'] = df5['clean_tweet'].apply(len)

In [106]:
df5

,userid,account_type,clean_tweet,bert_clear_expression_conf,lang,vader_sentiment,vader_label,url_flag,len
0,787405734442958848,bot,"We can put the ""River"" in ""Screwdrivers"" https...",90,en,0.0000,neutral,True,64
1,787405734442958848,bot,"You can't extract the ""Pong"" from ""Sponged"" ht...",98,en,0.0000,neutral,True,67
2,787405734442958848,bot,"Put the ""Nope"" back in ""Inoperative"" https://t...",97,en,0.0000,neutral,True,60
3,787405734442958848,bot,"We can't spell ""Artworks"" without ""Two"" https:...",85,en,0.0000,neutral,True,63
4,787405734442958848,bot,"You can be the ""She"" in ""Astonished"" https://t...",51,en,0.0000,neutral,True,60
...,...,...,...,...,...,...,...,...,...
488632,3077047801,bot,A Tribute – Joan Rivers on Fitness http://t.co...,69,en,0.2732,positive,True,74
488633,3077047801,bot,Live in Orlando area? Want a personal trainer?...,87,en,0.1680,positive,False,143
488634,3077047801,bot,If you're trying to get fit keep working. You...,83,en,0.8608,positive,True,138
488635,3077047801,bot,Ready for summer? Cheap cool sunglasses - loo...,96,en,0.7430,positive,True,133


In [117]:
df5.to_csv('tweets_f234_users_vader_bert_url_len.csv', index = False)

In [107]:
clf = RandomForestClassifier(random_state=0)

In [108]:
X = df5[['vader_sentiment', 'url_flag', 'len', 'lang', 'bert_clear_expression_conf']]
y = df5['account_type']

In [111]:
X = pd.get_dummies(X, columns = ['lang'], drop_first = False)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [113]:
clf = clf.fit(X_train, y_train)

In [114]:
pred = clf.predict(X_test)

In [115]:
labels = ['bot', 'human']

In [116]:
print(classification_report(y_test, y_pred = pred, labels = labels))

              precision    recall  f1-score   support

         bot       0.63      0.51      0.57     38217
       human       0.86      0.91      0.88    123034

    accuracy                           0.81    161251
   macro avg       0.75      0.71      0.72    161251
weighted avg       0.80      0.81      0.81    161251

